This module contains a model to solve microeconomic dynamic stochastic optimization problem, as described in Section 9 of Chris Carroll's SolvingMicroDSOPs.pdf notes. 

The model tries to match the age-conditional wealth profile of simulated consumers to the median wealth holdings of seven age groups in the 2004 SCF by varying only two parameters: the coefficient of relative risk aversion and a scaling factor for an age-varying sequence of discount factors.

The model uses a consumption-saving model with idiosyncratic shocks to permanent and transitory
income as defined in $\texttt{ConsIndShockModel}$.

In [ ]:
# Initial imports and notebook setup, click arrow to show
%matplotlib inline
import matplotlib.pyplot as plt

import HARK.ConsumptionSaving.ConsIndShockModel as Model # The consumption-saving micro model
import HARK.SolvingMicroDSOPs.SetupSCFdata as Data                               # SCF 2004 data on household wealth
import HARK.SolvingMicroDSOPs.EstimationParameters as Params           # Parameters for the consumer type and the estimation
from HARK.utilities import plotFuncsDer, plotFuncs

from HARK.simulation import drawDiscrete         # Method for sampling from a discrete distribution
from HARK.estimation import minimizeNelderMead, bootstrapSampleFromData # Estimation methods

In [ ]:
# Set up default values for CRRA and DiscFac
Params.init_consumer_objects["CRRA"]= 2.00            # Default coefficient of relative risk aversion (rho)
Params.init_consumer_objects["DiscFac"]= 0.97         # Default intertemporal discount factor (beta)
Params.init_consumer_objects["PermGroFacAgg"]= 1.0    # Aggregate permanent income growth factor (only matters for simulation)
Params.init_consumer_objects["aNrmInitMean"]= 0.0
Params.init_consumer_objects["aNrmInitStd"]= 1.0
Params.init_consumer_objects["pLvlInitMean"]= 0.0
Params.init_consumer_objects["pLvlInitStd"]= 0.0

In [ ]:
# Make a lifecycle consumer to be used for estimation
ThisType = Model.IndShockConsumerType(**Params.init_consumer_objects)

In [ ]:
# Solve and simulate the model
ThisType.solve()
ThisType.unpackcFunc()

ThisType.T_sim = 120
#ThisType.track_vars = ['mNrmNow','cNrmNow','pLvlNow','t_age']
ThisType.initializeSim()
ThisType.simulate()

In [ ]:
# Plot the consumption functions during working life
print('Consumption functions while working:')
mMin = min([ThisType.solution[t].mNrmMin for t in range(ThisType.T_cycle)])
plotFuncs(ThisType.cFunc[:ThisType.T_retire],mMin,5)